### Utilisation de Google Colab

Si vous utilisez Google Colab, suivez les instructions ci-dessous.

Tout d'abord, sélectionnez l'option GPU de Colab avec *Edit > Notebook settings* et sélectionner GPU comme Hardware accelerator. Installer ensuite deeplib avec la commande suivante:

In [ ]:
!pip install git+https://github.com/ulaval-damas/glo4030-labs.git

# Laboratoire 8: Transformers

Dans ce laboratoire, nous allons implémenter l'architecture Transformer de "Attention Is All You Need" (Vaswani, 2017).

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pathlib
import math

import numpy as np

import torch
import torch.random
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T

import matplotlib.pyplot as plt

import einops as ein

import deeplib.training as dtrain
import deeplib.datasets as ddatasets

## Partie 1: Scaled Dot-Product Attention

Au cœur des transformers se trouve le principe d'attention, qui permet de se concentrer sur certaines parties de l'information plutôt que sur d'autres.
Dans le cas des transformers, on utilise le "Scaled Dot-Product Attention" décrit dans "Attention Is All You Need".
Ce principe fonctionne comme une requête sur une hashmap: on fait une requête $q$ (query) selon des clés $k_i$ (keys), et on reçoit la valeur $v_i$ (values) associée à $q$.
Plus formellement, on peut représenter la hashmap comme un ensemble de pair (clé, valeur) $D = \{(k_1, v_1), \ldots (k_m, v_m)\}$, où $k_i$ sont les clés et $v_i$ leur valeur associée.
Ainsi, lorsque l'on fait la requête $q$ sur la base de données $D$, on obtient la valeur associée à $q$.

L'attention est similaire à ce principle, sauf que $q$, $k_i$ et $v_i$ sont tous des vecteurs, et la réponse est une somme pondérée de $v_i$.
Ainsi, l'attention calcule la fonction suivante:
$$Attention(q, D) = \sum_{i=1}^{m}{\alpha(q, k_i)v_i}$$

L'attention portée à une valeur est donnée par $\alpha(q, k_i)$.
Il s'agit en quelque sorte d'une mesure de la similarité entre la requête $q$ et les clés $k_i$.
Comme le nom "Scaled Dot-Product Attention" laisse deviner, cette similarité, $\alpha(\cdot)$, est un produit scalaire entre les vecteurs $q$ et $k_i$:

$$\alpha(q, k_i) = \frac{qk_i^T}{\sqrt{d}} = \frac{q \cdot k_i }{\sqrt{d}}$$

$d$ est le nombre de dimension des vecteurs. On justifiera le facteur d'échelle de $\frac{1}{\sqrt{d}}$ plus tard dans le laboratoire.


On normalise souvent cette valeur à l'aide d'une softmax afin de garder chaque pondération positive et sommant à 1.

$$\alpha(q, k_i) = \frac{\exp(\alpha(q, k_i))}{\sum_j\exp(\alpha(q, k_j))}$$

Ainsi, le calcul d'attention dans un Transformer est:

$$\alpha(q, k_i) = \frac{\exp\left(q \cdot k_i / \sqrt{d}\right)}{\sum_j\exp\left(q \cdot k_i / \sqrt{d}\right)}$$

Ensuite, il est possible de prendre une somme pondérée des valeurs $v_i$ selon $\alpha(q, k_i)$:

$$Attention(q, D) = \sum_{i=1}^{m}{\alpha(q, k_i)v_i}$$

Dans la cellule suivante, remplisser la fonction suivante pour appliquer l'attention sur les tenseurs $q$, $k$ et $v$.
Les dimensions des tenseurs sont spécifiés en commentaire.
Utilisez `F.softmax` pour normaliser l'attention.

Assurez-vous que votre calcul d'attention donne bien la valeur attendue.

In [ ]:
def simple_attention(q, k, v):
    """
    Attention simple
    q est un tenseur de dimension (1, d)
    k est un tenseur de dimension (n, d)
    v est un tenseur de dimension (n, d)
    d est la dimension des vecteurs, n le nombre de clés et de valeurs
    La sortie devrait être de dimension (1, d)
    """
    # TODO calculer le scaled-dot product attention
    return
    
q = F.normalize(torch.tensor([[3.0, 3.0]]), dim=1)
k = F.normalize(torch.tensor([[2.0, 1.5], [-4, 3], [0, 3], [-1, -4]]), dim=1)
v = F.normalize(torch.tensor([[-1.0, 2], [3, 4], [-2, -1], [0, 2]]), dim=1)

expected = torch.tensor([[-0.3584,  0.4563]])
output = simple_attention(q, k, v)

print(f'simple_attention(q, k, v) =\t{output}')
print(f'valeur attendue =\t\t{expected}')

### Visualisation de l'attention
Le produit scalaire peut aussi s'écrire sous la forme $q\cdot k_i = \|q\|\|k_i\|\cos(\theta)$, où $\theta$ est l'angle entre les vecteurs.
Si on fait l'hypothèse que les vecteurs en jeux sont unitaires, alors on a que le produit scalaire entre deux donne une mesure d'à quel point les vecteurs sont alignés.
On utilise cette interprétation du produit scalaire pour mieux visualiser ce que fait le Scaled Dot-Product Attention.

Dans la cellule suivante, remplissez le `TODO` avec le calcul pour calculer `alpha`, le vecteur d'attention.

In [ ]:
def show_attention(q, k, v):
    # TODO calculer le vecteur d'attention, après la softmax
    d = q.size(-1)
    alpha = ...
    
    figs, axes = plt.subplots(ncols=2, figsize=(14, 6))
    ax = axes[0]
    ax.arrow(0, 0, q[0][0], q[0][1], head_width=0.1, head_length=0.1, fc='red', ec='red', label='$q$')

    colors = ['rosybrown', 'cornflowerblue', 'lightcoral', 'mediumpurple']
    for i, k_i in enumerate(k):
        ax.arrow(0, 0, k_i[0], k_i[1], head_width=0.1, head_length=0.1, label=f'$k_{i}$, $\\alpha(q, k_{i}) = {alpha[0][i]:.3f}$', fc=colors[i], ec=colors[i])

    ax.set_xlim(-1.25, 1.25)
    ax.set_ylim(-1.25, 1.25)
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    ax.set_title('$q$ and $k_i$')
    ax.legend(loc='lower right')
    ax.set_aspect('equal', adjustable='box')
    
    ax = axes[1]
    ax.arrow(0, 0, output[0][0], output[0][1], head_width=0.1, head_length=0.1, fc='red', ec='red', label='$\sum_i v_{i} \cdot \\alpha(q, k_{i})$')

    for i, v_i in enumerate(v):
        v_i = v_i * alpha[0, i]
        ax.arrow(0, 0, v_i[0], v_i[1], head_width=0.1, head_length=0.1, label=f'$v_{i} \cdot \\alpha(q, k_{i})$', fc=colors[i], ec=colors[i])

    ax.set_xlim(-1.25, 1.25)
    ax.set_ylim(-1.25, 1.25)
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    ax.set_title(r'Valeurs pondérées par $\alpha$')
    ax.legend(loc='lower right')
    ax.set_aspect('equal', adjustable='box')

    plt.show()
    
show_attention(q, k, v)

#### Questions
- Visuellement, quel clé est la plus alignée avec $q$? Que peut-on dire de la valeur de $\alpha$ correspondante? Qu'est-ce que cela veut-dire à propose de la valeur finale?
- Que ce passe-t'il si vous changez tous les $k_i$ afin qu'ils soient opposés à $q$, sauf un? Quel est l'impact sur la valeur retournée par l'attention?
- Que ce passe-t'il si vous changez tous les $k_i$ afin qu'ils soient opposés à $q$, sauf **deux** qui ont des valeurs égales? Quel est l'impact sur la valeur retournée par l'attention?

### Plusieurs requêtes

Maintenant, on veut implémenter l'attention avec plusieurs requêtes $q_i$.
On veut comparer chaque $q_i$ avec chaque $k_i$.
Afin d'accélérer le calcul de l'attention, on va représenter cette opération grâce à une multiplication de tenseurs.

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d}}\right)V$$

$$Q=\begin{pmatrix}
q_1\\
q_2\\
\vdots\\
q_n
\end{pmatrix}_{n\times d}, K=\begin{pmatrix}
k_1\\
k_2\\
\vdots\\
k_n
\end{pmatrix}_{n\times d}, V=\begin{pmatrix}
v_1\\
v_2\\
\vdots\\
v_n
\end{pmatrix}_{n\times d}$$

Chaque $q_i$, $k_i$ et $v_i$ sont des vecteurs de dimensions $d$.

La comparaison de chaque $q_i$ et $k_i$ se fait lors du produit suivant:

$$QK^T = \begin{pmatrix}
q_1\\
q_2\\
\vdots\\
q_n
\end{pmatrix}_{n\times d} \begin{pmatrix}
k_1 & k_2 & \dots & k_n
\end{pmatrix}_{d\times n} = \begin{pmatrix}
q_1k_1 & q_1k_2 & \dots  & q_1k_n \\
q_2k_1 & q_2k_2 & \dots  & q_2k_n \\
\vdots & \vdots & \ddots & \vdots \\
q_nk_1 & q_nk_2 & \dots  & q_nk_n \\
\end{pmatrix}_{n\times n}$$

On appelle $QK^T$ la matrice d'attention.
Chaque élément de cette matrice correspond au produit scalaire entre une requête et une clé.

Si on ignore la softmax et le facteur d'échelle, on obtient bel et bien une somme pondérée des valeurs, selon la similarité entre les clés et les requêtes.

$$QK^TV = \begin{pmatrix}
q_1k_1 & q_1k_2 & \dots  & q_1k_n \\
q_2k_1 & q_2k_2 & \dots  & q_2k_n \\
\vdots & \vdots & \ddots & \vdots \\
q_nk_1 & q_nk_2 & \dots  & q_nk_n \\
\end{pmatrix}_{n\times n}\begin{pmatrix}
v_1\\
v_2\\
\vdots\\
v_n
\end{pmatrix}_{n\times d}=\begin{pmatrix}
q_1k_1v_1 + q_1k_2v_2 + \dots  + q_1k_nv_n \\
q_2k_1v_1 + q_2k_2v_2 + \dots  + q_2k_nv_n \\
\vdots \\
q_nk_1v_1 + q_nk_2v_2 + \dots  + q_nk_nv_n \\
\end{pmatrix}_{n\times d}$$

Dans la cellule suivante, implémenter l'attention en batch avec plusieurs requêtes.
Les dimensions des tenseurs sont spécifiés en commentaire.

Assurez-vous que votre calcul d'attention donne bien la valeur attendue.

In [ ]:
def scaled_dot_product_attention(q, k, v, mask=None):
    """
    Attention sur batch
    q est un tenseur de dimension (b, n, d)
    k est un tenseur de dimension (b, n, d)
    v est un tenseur de dimension (b, n, d)
    b est la taille de la mini-batch, n le nombre de clés et de valeurs, d est la dimension des vecteurs
    La sortie devrait être de dimension (b, n, d)
    """
    # TODO
    attention = ...
    if mask is not None:
        # NE PAS FAIRE TOUT DE SUITE, voir plus bas.
        # Commencez par calculer la matrice d'attention sans le masque
        # TODO appliquer un masque sur `attention`, détails dans la cellule suivante
        ...
    
    # TODO calculer la somme pondérée de v selon attention
    return

q = F.normalize(torch.tensor([[3.0, 3.0], [-1, 3.2], [0.2, -0.4], [0, -1]]), dim=1).unsqueeze(0)
k = F.normalize(torch.tensor([[2.0, 1.5], [-4, 3], [0, 3], [-1, -4]]), dim=1).unsqueeze(0)
v = F.normalize(torch.tensor([[-1.0, 2], [3, 4], [2, -3], [0, 2]]), dim=1).unsqueeze(0)

expected = torch.tensor([[[0.1089, 0.3322], [0.2860, 0.2639], [0.0600, 0.6791], [0.0986, 0.7087]]])
output = scaled_dot_product_attention(q, k, v)

print(f'scaled_dot_product_attention(q, k, v) =\n{output}')
print(f'valeur attendue =\n{expected}')

#### Questions
- Quelle est la taille du tenseur lorsque l'on multiple $q$ et $v$? Est-ce problématique pour le temps de calculs et la mémoire utilisée si $n$ (le nombre de queries, keys et valeurs) augmente?


#### Masques
Implémentez l'opération de masque dans la fonction `scaled_dot_product_attention`.
Le masque permet d'empêcher certaines requêtes de porter attention à certaines valeurs.

Ceci est possible en remplaçant les éléments de la matrice d'attention par $-\infty$ où `mask` est `False`

In [ ]:
q = F.normalize(torch.tensor([[3.0, 3.0], [-1, 3.2], [0.2, -0.4], [0, -1]]), dim=1).unsqueeze(0)
k = F.normalize(torch.tensor([[2.0, 1.5], [-4, 3], [0, 3], [-1, -4]]), dim=1).unsqueeze(0)
v = F.normalize(torch.tensor([[-1.0, 2], [3, 4], [2, -3], [0, 2]]), dim=1).unsqueeze(0)
mask = torch.tensor([[1, 0, 0, 0], [1, 1, 0, 0], [1, 1, 1, 0], [1, 1, 1, 1]], dtype=torch.bool)

expected = torch.tensor([[[-0.4472,  0.8944], [ 0.1639,  0.8393], [ 0.1129,  0.3967], [ 0.0986,  0.7087]]])
output = scaled_dot_product_attention(q, k, v, mask)

print(f'scaled_dot_product_attention(q, k, v) =\n{output}')
print(f'valeur attendue =\n{expected}')

#### Visualisation de l'attention
Il est aussi possible de visualiser directement la matrice d'attention.

Dans la cellule suivate, calculez l'attention avec un masque.

In [ ]:
q = F.normalize(torch.tensor([[3.0, 3.0], [-1, 3.2], [0.2, -0.4], [0, -1]]), dim=1).unsqueeze(0)
k = F.normalize(torch.tensor([[2.0, 1.5], [-4, 3], [0, 3], [-1, -4]]), dim=1).unsqueeze(0)
v = F.normalize(torch.tensor([[-1.0, 2], [3, 4], [2, -3], [0, 2]]), dim=1).unsqueeze(0)
mask = torch.tensor([[1, 0, 0, 0], [1, 1, 0, 0], [1, 1, 1, 0], [1, 1, 1, 1]], dtype=torch.bool)
# mask = torch.ones((4, 4), dtype=torch.bool)

# TODO
attention = ...
softmax = ...
output = ...

qs = [f'$q_{i}$' for i in range(q.size(1))]
ks = [f'$k_{i}$' for i in range(q.size(1))]

plt.imshow((softmax[0]))

plt.xticks(range(q.size(1)), ks)
plt.yticks(range(q.size(1)), qs)

plt.xlabel('Keys')
plt.ylabel('Queries')

plt.colorbar()
plt.show()

#### Questions
- Que se passe-t'il lors de l'ajout d'un masque?

- Pour les questions suivantes, n'utilisez pas de masque.

- Que se passe-t'il si on change l'ordre des requêtes $q$? Vous pouvez changer l'ordre manuellement avec `q = q[:, [1, 0, 2, 3]]`.

- Que se passe-t'il si on change l'ordre des clés et des valeurs? Vous devez permuter $k$ et $v$ de la même façon.

- Est-ce l'invariance à l'ordre des entrées est problématique? Quant est-t'il si l'ordre est important? par exemple en traitement de la langue naturelle.


### Encodage de position
Comme on vient de le remarquer, les Transformers sont invariants à la permutation des $q_i$ et des paires $(k_i, v_i)$, on doit donc encoder l'information sur la position directement dans ces vecteurs.
Pour ce faire, on utilise une suite de sinus et de cosinus.

L'idée est de générer des vecteurs où chaque composante oscille à des fréquences différentes.
Comme illustré dans la figure suivante issue de [Visual Guide to Transformer Neural Networks, Hedu AI](https://www.youtube.com/watch?v=dichIcUZfOw), chaque vecteur (une slice verticale dans le graphique) permet de représenter une position dans la séquence.
Les composantes avec une fréquence faible (en bleu) permettent d'avoir un estimé de la position sur l'ensemble de la séquence, mais avec peu de précision.
Les composantes avec une fréquence élevée (en rouge) permettent d'avoir une estimation plus précise de la position, mais seulement sur une petite partie de la séquence puisque les valeurs se répètent périodiquement.

![Positional Encoding](docs/pos_encoding.png)

Dans la cellule suivante, implémenter un encodage de position similaire à ce qui est utilisé dans "Attention Is All You Need".
Il faudra générer un tenseur de dimensions $(1, n, d)$, où $n$ est le nombre d'éléments et $d$ le nombre de dimension de $q_i$.
Générez l'encodage selon cet encodage:

$$\text{PE}_{pos, 2i} = \sin(\text{pos} / 10000^{i/d})$$

$$\text{PE}_{pos, 2i + 1} = \cos(\text{pos} / 10000^{i/d})$$

Où $\text{pos}$ est la position dans la séquence (de 0 à $n - 1$) et $i$ est l'index dans le vecteur (de 0 à $d - 1$).

In [ ]:
def position_encoding(n_keys: int, dimension: int, device: torch.device = torch.device("cpu")):
    pos = torch.arange(n_keys, dtype=torch.float, device=device).reshape(1, -1, 1)
    dim = torch.arange(dimension, dtype=torch.float, device=device).reshape(1, 1, -1)
    # TODO
    phase = ...

    return torch.where(dim.long() % 2 == 0, torch.sin(phase), torch.cos(phase))

pe = position_encoding(64, 128)
plt.imshow(pe.squeeze())
plt.xlabel('dimension')
plt.ylabel('n')
plt.show()

Dans la cellule suivante, ajoutez de l'encodage de position sur $q$ et $k$.

In [ ]:
q = F.normalize(torch.tensor([[3.0, 3.0], [-1, 3.2], [0.2, -0.4], [0, -1]]), dim=1).unsqueeze(0)
k = F.normalize(torch.tensor([[2.0, 1.5], [-4, 3], [0, 3], [-1, -4]]), dim=1).unsqueeze(0)
v = F.normalize(torch.tensor([[-1.0, 2], [3, 4], [2, -3], [0, 2]]), dim=1).unsqueeze(0)

_, n, d = q.shape

# TODO
q = ...
k = ...

scaled_dot_product_attention(q, k, v)

#### Questions
- Maintenant, que se passe-t'il lorsque l'on permute les $q_i$? et les $(k_i, v_i)$ après l'ajout de l'encodage de position?


### Le "Scaled" dans Scaled Dot-Product Attention

Le facteur de $\frac{1}{\sqrt{d}}$ peut sembler inutile, pourtant, il est crucial à l'optimisation des Transformers.
Si on fait l'hypothèse que $q_i$ et $k_i$ suivent une distribution normale $\mathcal{N}(0, 1)$, le produit entre $q$ et $k$ aura une variance de $d$.
Il faut donc diviser par $\sqrt{d}$ afin d'obtenir une variance de $1$.

Dans la cellule suivante, calculez `attention` en faisant le produit entre $q$ et $k$ sans le facteur d'échelle.
Ensuite, calculez `scaled_attention` en y ajoutant le facteur de $\frac{1}{\sqrt{d}}$.

Comparez la variance dans les deux cas.

In [ ]:
torch.manual_seed(1337)
num_tokens = 32
dimension = 256

q = torch.randn(num_tokens, dimension)
k = torch.randn(num_tokens, dimension)

# TODO
attention = ...
# TODO
scaled_attention = ...

print(f'Variance sans scaling: {attention.var()}')
print(f'Variance avec scaling: {scaled_attention.var()}')

#### Questions
- Justifiez pourquoi la variance est de $d$


#### Impact sur les Transformers

Quel est l'impact d'une grande variance sur la softmax?

Comparez la sortie de la softmax des vecteurs `attention` et `scaled_attention` de la cellule précédente.

In [ ]:
# TODO
low_variance_softmax = ...
high_variance_softmax =  ...

print(f'Softmax avec faible variance:\n\t{[round(x.item(), 5) for x in low_variance_softmax]}')
print(f'Softmax avec haute variance:\n\t{[round(x.item(), 5) for x in high_variance_softmax]}')

#### Questions
- Que remarquez-vous du vecteur softmax lorsque la variance est grande?

- Pourquoi est-ce que ceci serait mauvais le Transformer?


## Partie 2: Multi-Head Attention
On va maintenant créer un module PyTorch utilisant l'attention.

### AttentionHead
On commence par définir `AttentionHead`, un module représentant une tête d'attention à l'intérieur d'un Transformer.
- On projette `q`, `k`, et `v` avec des couches linéaires sans biais.
- On applique du dropout sur la matrice d'attention après la softmax.

Remplissez les TODOs dans le code suivant.

In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, dim_embed: int, head_size: int, dropout: float):
        super().__init__()
        # TODO trois couches linéaires sans biais
        self.q_proj = ...
        self.k_proj = ...
        self.v_proj = ...
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, q, k, v, mask=None):
        B, T, C = q.shape
        # TODO appliquer les couches lineaires  à q, k et v
        q = ...
        k = ...
        v = ...
        
        # TODO calculer la matrice d'attention
        attention = ...
        
        # TODO
        if mask is not None:
            # Appliquer le masque
            ...
    
        softmax = ...
        
        # Appliquer du dropout après la softmax
        softmax = self.dropout(softmax)
        
        # TODO
        return

    
head = AttentionHead(16, 32, 0.1)
x = torch.randn((1, 6, 16))
output = head(x, x, x)

print(f'Shape \t\t= {tuple(output.shape)}')
print(f'Expected shape \t= {(1, 6, 32)}')

### Multi-Head Attention
On définit maintenant le module `MultiHeadAttention` qui utilise plusieurs têtes d'attention séparées.
- Plusieurs têtes d'attention.
- Concaténation des sorties de chaque tête
- Couche linéaire
- Dropout

Remplissez les TODOs dans le code suivant.

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads: int, dim_embed: int, head_size: int, dropout: float):
        super().__init__()
        # TODO créer une liste de `num_heads` modules AttentionHead
        self.heads = nn.ModuleList(...)
        self.fc = nn.Linear(num_heads * head_size, dim_embed)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, q, k, v, mask=None):
        out = torch.cat([h(q, k, v, mask) for h in self.heads], dim=-1)
        # TODO appliquer fc suivit de dropout
        return

    
m_head = MultiHeadAttention(3, 32, 256, 0.1)
x = torch.randn((1, 6, 32))
output = m_head(x, x, x)

print(f'Shape \t\t= {tuple(output.shape)}')
print(f'Expected shape \t= {(1, 6, 32)}')

## Partie 3: TransformerEncoderLayer

### FeedForward
On définit maintenant le réseau `FeedForward` utilisé dans le Transformer.

- Linear: dim_model -> 4 * dim_model
- ReLU
- Linear: 4 * dim_model -> dim_model
- Dropout

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, dim_model: int, dropout: float):
        super().__init__()
        # TODO Créer le réseau selon les indications de la cellule précédente
        self.ffn = nn.Sequential(
            ...
        )
        
    def forward(self, x):
        return self.ffn(x)


### TransformerEncoderLayer
On peut maintenant construire une couche du Transformer.

- LayerNorm
- MultiHeadAttention
- Connexion résiduelle
- LayerNorm
- FeedForward
- Connexion résiduelle

In [ ]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, dim_model: int, num_head: int, dropout: float):
        super().__init__()
        head_size = dim_model // num_head
        self.attention = MultiHeadAttention(num_head, dim_model, head_size, dropout)
        self.ffn = FeedForward(dim_model, dropout)
        self.norm1_q = nn.LayerNorm(dim_model)
        self.norm1_k = nn.LayerNorm(dim_model)
        self.norm1_v = nn.LayerNorm(dim_model)
        self.norm2 = nn.LayerNorm(dim_model)
        
    def forward(self, q, k, v, mask=None):
        # TODO appliquer LayerNorm à q, k et v
        q = ...
        k = ...
        v = ...
        
        q = q + self.attention(q, k, v, mask)
        q = q + self.ffn(self.norm2(q))
        return q
    
transf = TransformerEncoderLayer(64, 2, 0.1)
x = torch.randn((1, 6, 64))
output = transf(x, x, x)

print(f'Shape \t\t= {tuple(output.shape)}')
print(f'Expected shape \t= {(1, 6, 64)}')

## Partie 4: TransformerEncoder
On peut maintenant tout mettre ensemble et construire le `TransformerEncoder`!

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, num_layers: int, dim_model: int, num_heads: int, out_size: int, dropout: float):
        super().__init__()
        # TODO Créer une liste de `num_layers` modules TransformerEncoderLayer
        self.layers = nn.ModuleList(...)
        self.norm = nn.LayerNorm(dim_model)
        self.fc = nn.Linear(dim_model, out_size)
        
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        
    def forward(self, x, mask=None):
        batch_size, num_tokens, dim = x.shape
        # TODO ajouter l'encodage de position avec `position_encoding`
        x += ...
        
        for layer in self.layers:
            # TODO appeler layer avec x comme valeur de q, k et v, ne pas oublier de passer le mask
            x = ...
            
        x = self.norm(x)
        x = self.fc(x)
        
        return x
    
x = torch.randn((2, 64, 64))

transformer_enc = TransformerEncoder(6, 64, 2, 10, 0.0)
output = transformer_enc(x)

print(f'Shape \t\t= {tuple(output.shape)}')
print(f'Expected shape \t= {(2, 64, 10)}')

## Partie 5: Entraînement
On entraîne maintenant notre Transformer à prédire du texte.
- Inspiré par https://github.com/karpathy/ng-video-lecture/tree/master de Andrej karpathy


In [ ]:
# Télécharger les données localement
!curl -o ~/GLO-4030/datasets/tiny_shakespeare.txt https://raw.githubusercontent.com/karpathy/ng-video-lecture/master/input.txt

In [ ]:
# Sur Google Colab
import requests

url = 'https://raw.githubusercontent.com/karpathy/ng-video-lecture/master/input.txt'
res = requests.get(url)
text = res.content.decode('utf-8')

# Extrait des données
print(text[:147])

In [ ]:
# Sur Calcul Québec et localement
# Charger les données
data_path = pathlib.Path('~/GLO-4030/datasets/tiny_shakespeare.txt').expanduser()
with open(data_path, 'r') as f:
    text = f.read()

# Extrait des données
print(text[:147])

In [ ]:
# Calculer le vocabulaire (les caractères utilisés dans le texte)
characters = sorted(list(set(text)))
vocab_size = len(characters)

str_to_int = {c:i for i, c in enumerate(characters)}
int_to_str = {v: k for k, v in str_to_int.items()}

# Fonction utilitaires pour convertir les caractères en entier, et inversement
def encode(s):
    return [str_to_int[c] for c in s]

def decode(s):
    return [int_to_str[c] for c in s]

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
batch_size = 128

# Générer une batch aléatoire
def get_batch(split, block_size, device):
    data = train_data if split == 'train' else val_data
    indices = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in indices])
    y = torch.stack([data[i+1:i+block_size+1] for i in indices])
    x, y = x.to(device), y.to(device)
    return x, y

print(get_batch('train', 256, 'cpu'))

In [ ]:
class TransformerNLP(nn.Module):
    def __init__(self, num_layers: int, dim_model: int, num_heads: int, out_size: int, dropout: float, vocab_size: int, block_size: int):
        super().__init__()
        # Embedding pour convertir des entiers en vecteurs
        # Il s'agit essentiellement d'un tableau associant chaque entier à un vecteur
        self.embedding = nn.Embedding(vocab_size, dim_model)
        # Notre transformer!
        self.transformer = TransformerEncoder(num_layers, dim_model, num_heads, out_size, dropout)
        # Masque pour éviter que les lettre puissent porter attention aux lettres subséquentes
        # On ne veut permettre aux lettres que de porter attention aux lettres précédentes
        # On utilise `tril` afin de générer une matrice triangulaire inférieure
        self.register_buffer('mask', torch.tril(torch.ones(block_size, block_size)))
        
    def forward(self, x):
        B, T = x.shape
        # TODO appliquer l'embedding
        x = ...
        
        # TODO encodage de position
        x = ...
        
        # TODO passer x dans le transformer
        x = ...
        return x
        
    def loss(self, pred, y):
        B, T, C = pred.shape
        pred = pred.view(B * T, C)
        y = y.view(B * T)
        return F.cross_entropy(pred, y)
    
    def generate(self, idx, max_new_tokens, block_size):
        # Génère du texte avec le modèle
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [ ]:
block_size = 256
max_iters = 10_000
eval_interval = 1000
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.1

# Model
model = TransformerNLP(n_layer, n_embd, n_head, vocab_size, dropout, vocab_size, block_size)
# Load a pretrained model
model.load_state_dict(torch.load('checkpoints/transformer_nlp.pth'))
model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
iter_loss = []
for i in range(max_iters):
    if i % eval_interval == 0 or i == max_iters - 1:
        with torch.no_grad():
            out = {}
            model.eval()
            for split in ['train', 'val']:
                losses = torch.zeros(eval_iters)
                for k in range(eval_iters):
                    x, y = get_batch(split, block_size, device)
                    pred = model(x)
                    loss = model.loss(pred, y)
                    losses[k] = loss.item()
                out[split] = losses.mean()
            print(f"step {i}: train loss {out['train'].item():.4f}, val loss {out['val'].item():.4f}")
            iter_loss.append((out['train'].item(), out['val'].item()))
            model.train()
        x, y = get_batch('train', block_size, device)
        
        pred = model(x)
        loss = model.loss(pred, y)
        
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

In [ ]:
iter_loss_np = np.array(iter_loss)
plt.plot(iter_loss_np[:, 0], label='train')
plt.plot(iter_loss_np[:, 1], label='valid')
plt.legend()
plt.show()

In [ ]:
# Prediction du texte
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(''.join(decode(model.generate(context, 500, block_size)[0].tolist())))

In [ ]:
# Sauvegarde des poids
torch.save(model.state_dict(), pathlib.Path('~/GLO-4030/datasets/transformer_nlp.pth').expanduser())

## Partie 6: Vision Transformer

Les Transformers ne sont pas limités au traitement de la langue naturelle.
Plutôt que de passer des lettres ou des mots, on peut passer des portions d'images encodés en vecteur.

Pour cette partie, nous allons entraîner un ViT (Vision Transformer) sur CIFAR10.


In [ ]:
import torchvision.models

import poutyne as pt

In [ ]:
# Data
batch_size = 128

transform_train = T.Compose([
    T.RandAugment(4, 14),
    T.RandomCrop(32, padding=4),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = T.Compose([
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

cifar, cifar_test = ddatasets.load_cifar10()
cifar.transform = transform_train
cifar_test.transform = transform_test

train_split = 0.8
num_data = len(cifar)
indices = np.arange(num_data)
np.random.shuffle(indices)

split = math.floor(train_split * num_data)
train_idx, valid_idx = indices[:split], indices[split:]

train_dataset = torch.utils.data.Subset(cifar, train_idx)
valid_dataset = torch.utils.data.Subset(cifar, valid_idx)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(cifar_test, batch_size=batch_size)

In [ ]:
pathlib.Path('logs').mkdir(exist_ok=True, parents=True)

epoch = 100
learning_rate = 1e-4

img_size = cifar[0][0].shape[-1]
patch_size = 4
num_layers = 12
num_heads = 12
hidden_dim = 768
mlp_dim = 3072

model = torchvision.models.VisionTransformer(img_size, patch_size, num_layers, num_heads, hidden_dim, mlp_dim, dropout=0.1, attention_dropout=0.1, num_classes=10)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

model = pt.Model(model, optimizer, 'cross_entropy', batch_metrics=['accuracy'])
model.cuda()

history = model.fit_generator(train_loader, valid_loader, epochs=epoch, callbacks=[
    pt.ModelCheckpoint('logs/best_epoch_{epoch}.ckpt', monitor='val_acc', mode='max', save_best_only=True, restore_best=True, verbose=True, temporary_filename='best_epoch.ckpt.tmp'),
])

dtrain.History(history).display()

test_loss, test_acc = model.evaluate_generator(test_loader)
print('test_loss: {:.4f} test_acc: {:.2f}'.format(test_loss, test_acc))

In [ ]:
model.load_weights('logs/best_epoch_5.ckpt')
test_loss, test_acc = model.evaluate_generator(test_loader)
print('Epoch 5: test_loss: {:.4f} test_acc: {:.2f}'.format(test_loss, test_acc))

## Extra

### Transformers annotés

On vous recommande de consulter [willGuimont/transformers](https://github.com/willGuimont/transformers) pour mieux comprendre les modèles de Transformer.
Le projet implémente plusieurs architectures basées sur les Transformers; le code est bien commenté et devrait être facile à suivre.


### Self-attention vs Cross-attention

Dans les exemples utilisés dans ce laboratoire, $q$, $k$ et $v$ sont dérivés d'une seule valeur (`x = layer(x, x, x, mask)` dans le `TransformerEncoder`.
Ceci est appelé du "self-attention".

Il est aussi possible de faire du "cross-attention", lorsque $q$ et $(k, v)$ proviennent de données différentes.

[Perceiver](https://www.deepmind.com/publications/perceiver-general-perception-with-iterative-attention) et [PerceiverIO](https://www.deepmind.com/open-source/perceiver-io) sont de beaux exemples d'utilisation du "cross-attention".


### TransformerDecoder

Voici une implémentation du TransformerDecoder de "Attention Is All You Need".


In [ ]:
class TransformerDecoderLayer(nn.Module):
    def __init__(self, dim_model: int, num_head: int, dropout: float):
        super().__init__()
        head_size = dim_model // num_head
        self.self_attention = MultiHeadAttention(num_head, dim_model, head_size, dropout)
        self.norm1_target = nn.LayerNorm(dim_model)
        self.norm1_memory = nn.LayerNorm(dim_model)
        
        self.norm2 = nn.LayerNorm(dim_model)
        self.cross_attention = MultiHeadAttention(num_head, dim_model, head_size, dropout)
        
        self.norm3 = nn.LayerNorm(dim_model)
        self.dropout = nn.Dropout(dropout)
        self.ffn = FeedForward(dim_model, dropout)
    
    def forward(self, target, memory, self_attention_mask=None, cross_attention_mask=None):
        target, memory = self.norm1_target(target), self.norm1_memory(memory)
        target = target + self.self_attention(target, target, target, self_attention_mask)
        target = target + self.cross_attention(self.norm2(target), memory, memory, cross_attention_mask)
        return target + self.ffn(self.norm3(target))
    
class TransformerDecoder(nn.Module):
    def __init__(self, num_layers: int, dim_model: int, num_heads: int, out_size: int, dropout: float):
        super().__init__()
        self.layers = nn.ModuleList([TransformerDecoderLayer(dim_model, num_heads, dropout) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(dim_model)
        self.fc = nn.Linear(dim_model, out_size)
        
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            
    def forward(self, target, memory, self_attention_mask=None, cross_attention_mask=None):
        num_tokens, dim = target.shape[1], target.shape[2]
        target += position_encoding(num_tokens, dim, target.device)
        for layer in self.layers:
            target = layer(target, memory, self_attention_mask, cross_attention_mask)
        target = self.norm(target)
        target = self.fc(target)
        return target
    
dec = TransformerDecoder(6, 10, 5, 100, 0.0)
target = torch.randn((2, 5, 10))
memory = torch.randn((2, 5, 10))
print(dec(target, memory).shape)

### Exemple d'implémentation de ViT

In [ ]:
class VisionTransformer(nn.Module):
    def __init__(self, img_size: int, patch_size: int, num_classes: int, channels: int, num_layers: int, dim_model: int, num_heads: int, out_size: int, dropout: float):
        super().__init__()
        assert img_size % patch_size == 0, "img_size must be divisible by patch_size"
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        self.patch_dim = channels * patch_size * patch_size
        
        self.norm1 = nn.LayerNorm(self.patch_dim)
        self.fc1 = nn.Linear(self.patch_dim, dim_model)
        self.norm2 = nn.LayerNorm(dim_model)
        
        self.cls = nn.Parameter(torch.randn((1, 1, dim_model)))
        self.dropout = nn.Dropout(dropout)
        self.trans_enc = TransformerEncoder(num_layers, dim_model, num_heads, out_size, dropout)
        
        self.norm3 = nn.LayerNorm(dim_model)
        self.fc2 = nn.Linear(dim_model, num_classes)
    
    def forward(self, x):
        batch_size = x.shape[0]
        size = 8
        stride = 1
        
        # Image to patches
        x = self._img_to_patch_emdedding(x)
        
        # Add cls token
        cls = ein.repeat(self.cls, '1 1 d -> b 1 d', b=batch_size)
        x = torch.cat((cls, x), dim=1)
        
        # Positional encoding
        batch_size, num_tokens, dim = x.shape
        x += position_encoding(num_tokens, dim, x.device)
        
        # Dropout
        x = self.dropout(x)
        
        # Transformer
        x = self.trans_enc(x)

        
        # Final prediction from cls token
        x = x[:, 0]
        x = self.fc2(self.norm3(x))

        return x
    
    def _img_to_patch_emdedding(self, img):
        x = ein.rearrange(img, 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=self.patch_size, p2=self.patch_size)
        x = self.norm1(x)
        x = self.fc1(x)
        x = self.norm2(x)
        return x

In [ ]:
epoch = 100
batch_size = 256
learning_rate = 1e-4

img_size = 32
patch_size = 8
num_classes = 10
channels = 3
num_layers = 6
dim_model = 512
num_heads = 12
out_size = dim_model
dropout = 0.1

model = VisionTransformer(img_size, patch_size, num_classes, channels, num_layers, dim_model, num_heads, out_size, dropout)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

model = pt.Model(model, optimizer, 'cross_entropy', batch_metrics=['accuracy'])
model.cuda()

history = model.fit_generator(train_loader, valid_loader, epochs=epoch, callbacks=[
    pt.ModelCheckpoint('logs/vit_best_epoch_{epoch}.ckpt', monitor='val_acc', mode='max', save_best_only=True, restore_best=True, verbose=True, temporary_filename='best_epoch.ckpt.tmp'),
])

dtrain.History(history).display()

test_loss, test_acc = model.evaluate_generator(test_loader)
print('test_loss: {:.4f} test_acc: {:.2f}'.format(test_loss, test_acc))

### Attention comme passage de messages
- L'attention peut être interprété comme du passage de message sur un graphe.
- Chaque noeud $i$ dans le graphe possède un triplet $(q_i, k_i, v_i$)
- Grâce à $k_i$, chaque noeud "diffuse" ce qu'il connait
- Grâce à $q_i$, chaque noeud demande aux autres noeuds de l'information
- Selon ce que l'affinité entre les $q_i$ et les $k_i$, chaque noeud aggrège l'information $v_i$ de ses voisins
- Champ réceptif large, chaque noeud à accès aux autres noeuds en une seule opération.

<div>
<img src="docs/simplex_graph.png" width="500"/>
</div>

- Cette interprétation est cohérente avec la notion que les transformers sont des fonctions d'ensemble vers ensemble (invariance à l'ordre des éléments), la position des noeuds n'a pas d'importance, on doit encoder l'information sur la position autrement.